#Mounting Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Importing Libraries

In [ ]:
# Importing Libraries...
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

# Loading Data

In [ ]:
# Loading Data...
ratings = pd.read_csv('/content/drive/MyDrive/Semester-2/Recommendation System/Assignment_2/RAW_interactions.csv')
recipes = pd.read_csv('/content/drive/MyDrive/Semester-2/Recommendation System/Assignment_2/RAW_recipes.csv')

In [ ]:
ratings.head()

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."


In [ ]:
recipes.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [ ]:
# Checking for duplicates
print(ratings.duplicated().sum())
print(recipes.duplicated().sum())

0
0


In [ ]:
print(ratings.shape)
print(recipes.shape)

(1132367, 5)
(231637, 12)


# Preprocessing

In [ ]:
# Replacing column name of recipes dataframe...
recipes.rename(columns={'id':'recipe_id'},inplace=True)
recipes.head()

,name,recipe_id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


## Merging Data

In [ ]:
# Merging the ratings and recipes dataframe on recipe_id
ratings_with_name = ratings.merge(recipes, on='recipe_id')
ratings_with_name.head()

,user_id,recipe_id,date,rating,review,name,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...,white bean green chile pepper soup,495,1533,2002-09-21,"['weeknight', 'time-to-make', 'course', 'main-...","[204.8, 5.0, 9.0, 26.0, 24.0, 2.0, 10.0]",4,"['combine beans , onion , chilies , 1 / 2 teas...",easy soup for the crockpot.,"['great northern beans', 'yellow onion', 'dice...",9
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall...",white bean green chile pepper soup,495,1533,2002-09-21,"['weeknight', 'time-to-make', 'course', 'main-...","[204.8, 5.0, 9.0, 26.0, 24.0, 2.0, 10.0]",4,"['combine beans , onion , chilies , 1 / 2 teas...",easy soup for the crockpot.,"['great northern beans', 'yellow onion', 'dice...",9
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...,devilicious cookie cake delights,20,56824,2002-10-27,"['30-minutes-or-less', 'time-to-make', 'course...","[132.3, 11.0, 39.0, 5.0, 4.0, 11.0, 5.0]",5,"['blend together cake mix , oil and eggs', 'ad...",NaN,"[""devil's food cake mix"", 'vegetable oil', 'eg...",4
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...,baked potato toppings,10,64342,2004-02-25,"['15-minutes-or-less', 'time-to-make', 'course...","[2786.2, 342.0, 134.0, 290.0, 161.0, 301.0, 42.0]",3,['pick whichever topping you want to use and c...,these toppings sure makes a nice change from p...,"['mayonnaise', 'salsa', 'cheddar cheese', 'ref...",13
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin...",baked potato toppings,10,64342,2004-02-25,"['15-minutes-or-less', 'time-to-make', 'course...","[2786.2, 342.0, 134.0, 290.0, 161.0, 301.0, 42.0]",3,['pick whichever topping you want to use and c...,these toppings sure makes a nice change from p...,"['mayonnaise', 'salsa', 'cheddar cheese', 'ref...",13


## Filtering based on users

In [ ]:
# Filtering merged dataframe based on the users...
X = ratings_with_name.groupby('user_id').count()['rating'] > 100
genuine_users = X[X].index
genuine_users

Int64Index([      1533,       1535,       2310,       2312,       3288,
                  4291,       4439,       4470,       4740,       5060,
            ...
               2206637,    2324285,    2399085,    2549237,    2597942,
            1800054678, 1802849661, 2000431901, 2000498330, 2001102678],
           dtype='int64', name='user_id', length=1368)

In [ ]:
# Filtered ratings...
filtered_ratings = ratings_with_name[ratings_with_name['user_id'].isin(genuine_users)]

## Filtering Based on Recipes

In [ ]:
# Filtering Based on recipes...
y = filtered_ratings.groupby('name').count()['rating']>=50
famous_recipes = y[y].index

In [ ]:
# Final ratings dataframe...
final_ratings = filtered_ratings[filtered_ratings['name'].isin(famous_recipes)]

In [ ]:
# Dropping Duplicates if any...
final_ratings.drop_duplicates()

,user_id,recipe_id,date,rating,review,name,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
4212,149363,96886,2005-05-11,5,This was awesome! Our whole family loved this ...,kittencal s spinach parmesan rice bake,40,89831,2004-08-04,"['60-minutes-or-less', 'time-to-make', 'course...","[351.2, 36.0, 3.0, 20.0, 35.0, 70.0, 5.0]",7,"['set oven to 350f', 'generously grease a 11 x...",this is a wonderful easy side dish to chicken ...,"['frozen chopped spinach', 'cooked white rice'...",13
4214,209441,96886,2005-06-06,5,"Awesome IS the word for this, I made this for ...",kittencal s spinach parmesan rice bake,40,89831,2004-08-04,"['60-minutes-or-less', 'time-to-make', 'course...","[351.2, 36.0, 3.0, 20.0, 35.0, 70.0, 5.0]",7,"['set oven to 350f', 'generously grease a 11 x...",this is a wonderful easy side dish to chicken ...,"['frozen chopped spinach', 'cooked white rice'...",13
4216,73836,96886,2005-09-07,5,This was scrumptious. I halved the recipe suc...,kittencal s spinach parmesan rice bake,40,89831,2004-08-04,"['60-minutes-or-less', 'time-to-make', 'course...","[351.2, 36.0, 3.0, 20.0, 35.0, 70.0, 5.0]",7,"['set oven to 350f', 'generously grease a 11 x...",this is a wonderful easy side dish to chicken ...,"['frozen chopped spinach', 'cooked white rice'...",13
4217,160974,96886,2005-09-11,5,I'm a spinach lover but the rest of the family...,kittencal s spinach parmesan rice bake,40,89831,2004-08-04,"['60-minutes-or-less', 'time-to-make', 'course...","[351.2, 36.0, 3.0, 20.0, 35.0, 70.0, 5.0]",7,"['set oven to 350f', 'generously grease a 11 x...",this is a wonderful easy side dish to chicken ...,"['frozen chopped spinach', 'cooked white rice'...",13
4218,195175,96886,2005-09-15,5,This makes a great side dish for company. It's...,kittencal s spinach parmesan rice bake,40,89831,2004-08-04,"['60-minutes-or-less', 'time-to-make', 'course...","[351.2, 36.0, 3.0, 20.0, 35.0, 70.0, 5.0]",7,"['set oven to 350f', 'generously grease a 11 x...",this is a wonderful easy side dish to chicken ...,"['frozen chopped spinach', 'cooked white rice'...",13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1123827,130531,131018,2014-12-14,5,I have been making this recipe for a couple of...,pulled pork crock pot,1110,50509,2005-07-25,"['time-to-make', 'course', 'main-ingredient', ...","[335.1, 10.0, 72.0, 24.0, 80.0, 10.0, 8.0]",26,"['slice one onion and place in crock pot', 'pu...","i found this pulled pork recipe years ago, and...","['pork roast', 'onions', 'ginger ale', 'barbec...",4
1123829,121690,131018,2015-01-08,5,I&#039;ve made this twice without substitution...,pulled pork crock pot,1110,50509,2005-07-25,"['time-to-make', 'course', 'main-ingredient', ...","[335.1, 10.0, 72.0, 24.0, 80.0, 10.0, 8.0]",26,"['slice one onion and place in crock pot', 'pu...","i found this pulled pork recipe years ago, and...","['pork roast', 'onions', 'ginger ale', 'barbec...",4
1123855,51931,131018,2015-11-29,5,I am reviewing the recipe two ways. One as wr...,pulled pork crock pot,1110,50509,2005-07-25,"['time-to-make', 'course', 'main-ingredient', ...","[335.1, 10.0, 72.0, 24.0, 80.0, 10.0, 8.0]",26,"['slice one onion and place in crock pot', 'pu...","i found this pulled pork recipe years ago, and...","['pork roast', 'onions', 'ginger ale', 'barbec...",4
1123860,59686,131018,2016-03-18,5,"Since we butcher our own meat, I used a 4-1/2 ...",pulled pork crock pot,1110,50509,2005-07-25,"['time-to-make', 'course', 'main-ingredient', ...","[335.1, 10.0, 72.0, 24.0, 80.0, 10.0, 8.0]",26,"['slice one onion and place in crock pot', 'pu...","i found this pulled pork recipe years ago, and...","['pork roast', 'onions', 'ginger ale', 'barbec...",4


# Train-Test Split

In [ ]:
# Train-Test Split...
train_data, test_data = train_test_split(final_ratings, test_size=0.2)

In [ ]:
test_data

,user_id,recipe_id,date,rating,review,name,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
259366,28346,26205,2003-05-19,5,Made this for your Strawberry Rhubarb Pie #603...,classic pie crust idiot proof step by step ph...,55,102058,2002-04-24,"['60-minutes-or-less', 'time-to-make', 'course...","[2487.8, 225.0, 55.0, 49.0, 68.0, 367.0, 86.0]",24,"['preheat oven to 220c', 'put the flour , suga...",this pie crust retains the flavor of an all bu...,"['all-purpose flour', 'sugar', 'salt', 'unsalt...",7
249555,286566,105726,2008-08-29,5,We don't get Texas Toast here in Aus. so I bou...,kittencal s parmesan garlic bread or garlic toast,25,89831,2004-12-09,"['30-minutes-or-less', 'time-to-make', 'course...","[947.1, 42.0, 26.0, 64.0, 61.0, 79.0, 48.0]",13,"['set oven to 375f', 'for garlic bread', 'cut ...",for a nice change substitute finely shredded w...,"['french bread', 'butter', 'garlic', 'dried pa...",6
1041651,226372,66596,2009-03-16,5,Perfect! I used 3/4 cup of olive oil. Thanks...,kittencal s famous greek salad,10,89831,2003-07-14,"['15-minutes-or-less', 'time-to-make', 'course...","[489.7, 72.0, 23.0, 33.0, 15.0, 55.0, 4.0]",8,"['for the dressing', 'in a processor or use a ...",be prepared for the best greek salad on the pl...,"['olive oil', 'lemon, juice of', 'dried oregan...",16
939489,173314,69173,2007-05-20,5,Just made this in oven and will put in sauce t...,kittencal s italian melt in your mouth meatballs,50,89831,2003-08-20,"['60-minutes-or-less', 'time-to-make', 'course...","[1312.6, 129.0, 8.0, 108.0, 214.0, 174.0, 8.0]",5,['mix all ingredients together in a large bowl...,cooking the meatballs in simmering pasta sauce...,"['ground beef', 'egg', 'parmesan cheese', 'bre...",10
552887,55729,48760,2009-09-29,4,I do not often make Szechuan-style dishes (nor...,szechuan noodles with spicy beef sauce,30,37449,2002-12-13,"['30-minutes-or-less', 'time-to-make', 'course...","[617.1, 39.0, 41.0, 45.0, 63.0, 39.0, 21.0]",11,"['brown ground beef in a large skillet , stirr...",tired of using ground beef the same old way? t...,"['ground beef', 'onions', 'garlic', 'fresh gin...",12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191466,540346,106251,2015-03-01,5,This is my new favorite way to have cauliflowe...,roasted cauliflower 16 roasted cloves of garlic,40,58104,2004-12-16,"['60-minutes-or-less', 'time-to-make', 'course...","[126.9, 14.0, 11.0, 17.0, 6.0, 6.0, 3.0]",4,"['mix oil , rosemary , salt , pepper and garli...","i find it totally amazing, that children and m...","['cauliflower', 'garlic cloves', 'fresh rosema...",6
18769,574936,150384,2014-08-23,5,We don&#039;t know what Panera&#039;s soup tas...,panera broccoli cheese soup,70,192957,2006-01-04,"['time-to-make', 'course', 'main-ingredient', ...","[702.6, 81.0, 46.0, 33.0, 55.0, 162.0, 10.0]",14,"['saut onion in butter', 'set aside', 'cook me...",a very good broccoli cheese soup. served with ...,"['butter', 'onion', 'flour', 'half-and-half cr...",12
831211,628076,18914,2009-12-12,5,We had this with an Italian soup. The only cha...,incredibly delicious cheese garlic bread spread,30,27381,2002-02-05,"['30-minutes-or-less', '15-minutes-or-less', '...","[928.4, 138.0, 2.0, 59.0, 52.0, 156.0, 1.0]",5,['microwave margarine in medium glass bowl for...,"easy, cheesy, savory spread to transform regul...","['margarine', 'garlic', 'garlic powder', 'blac...",6
237402,153067,68955,2005-04-23,5,This meal was fabulous! I used drumsticks and...,japanese mum s chicken,45,29196,2003-08-13,"['60-minutes-or-less', 'time-to-make', 'course...","[313.1, 19.0, 54.0, 61.0, 62.0, 17.0, 5.0]",5,['place all the ingredients in a saucepan over...,"we have a japanese girl, tomoko, living with u...","['chicken drumsticks', 'water', 'balsamic vine...",7


# Creating Pivot Table

In [ ]:
# create pivot table for train data
train_pt = train_data.pivot_table(index='name', columns='user_id', values='rating')
# calculate similarity scores
similarity_scores = cosine_similarity(train_pt.fillna(0))
# create pivot table for test data
test_pt = test_data.pivot_table(index='name', columns='user_id', values='rating')
# test_pt.fillna(0,inplace=True)
# train_pt.fillna(0,inplace=True)

In [ ]:
test_pt

user_id,1535,2310,2312,3288,4291,4439,4470,4740,5060,6258,...,1926335,2123645,2206637,2324285,2549237,2597942,1800054678,1802849661,2000431901,2001102678
name,,,,,,,,,,,,,,,,,,,,,
24k carrots,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4 minute spicy garlic shrimp,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40 minute hamburger buns,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
absolute best ever lasagna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
all purpose stir fry sauce brown garlic sauce,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ww 0 point weight watchers cabbage soup,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
yes virginia there is a great meatloaf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
yummy crispy baked fish,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
test_pt[4470]['4 minute spicy garlic shrimp']

5.0

In [ ]:
47559 in train_pt.columns

True

In [ ]:
test_pt.columns

Int64Index([      1535,       2310,       2312,       3288,       4291,
                  4439,       4470,       4740,       5060,       6258,
            ...
               1926335,    2123645,    2206637,    2324285,    2549237,
               2597942, 1800054678, 1802849661, 2000431901, 2001102678],
           dtype='int64', name='user_id', length=1179)

# Recommendation Engine

In [ ]:
def get_top_k_recommendations(user_id, k, prev_item=None):
    if user_id not in train_pt.columns:
        return []
    # get previously recommended items
    prev_items = set()
    if prev_item is not None:
        prev_items.add(prev_item)

    rated_recipes = train_pt[user_id]
    rated_recipes = rated_recipes[rated_recipes.notna()]
    unrated_recipes = train_pt.index.difference(rated_recipes.index.union(prev_items))
    predicted_ratings = []

    for recipe_name in unrated_recipes:
        similarity_scores_for_recipe = similarity_scores[train_pt.index.get_loc(recipe_name)]
        user_ratings = train_pt[user_id].fillna(0)
        numerator = (user_ratings * similarity_scores_for_recipe).sum()
        denominator = similarity_scores_for_recipe.sum()

        if denominator == 0:
            predicted_rating = 0
        else:
            predicted_rating = numerator / denominator

        predicted_ratings.append((recipe_name, predicted_rating))

    top_recipe_names = [recipe_name for recipe_name, predicted_rating in sorted(predicted_ratings, key=lambda x: x[1], reverse=True)[:k]]
    return top_recipe_names


def display_recommendations(rec_list):
    print("Here are your recommendations:")
    for i, rec in enumerate(rec_list):
        print(f"{i+1}. {rec}")
    print("Enter the number of the recipe you liked, or '0' to exit:")


def get_user_feedback(rec_list):
    while True:
        display_recommendations(rec_list)
        selection = input()
        if selection == '0':
            return None
        try:
            selection_idx = int(selection) - 1
            if selection_idx < 0 or selection_idx >= len(rec_list):
                print("Invalid selection. Please try again.")
            else:
                return rec_list[selection_idx]
        except ValueError:
            print("Invalid selection. Please try again.")


def get_recipe_ingredients(recipe_name):
    recipe_row = final_ratings[final_ratings['name'] == recipe_name]
    ingredients_str = recipe_row['ingredients'].values[0]
    ingredients_list = [ing.strip() for ing in ingredients_str.split(',')]
    return ingredients_list


def main():
    user_id = int(input("Enter user id:"))
    prev_item = None
    while True:
        rec_list = get_top_k_recommendations(user_id, 5, prev_item)
        if not rec_list:
            print("No more recommendations available. Exiting...")
            return

        liked_item = get_user_feedback(rec_list)
        if liked_item is None:
            print("Exiting...")
            return
        prev_item = liked_item

        more_recommendations = input("Would you like more recommendations? (y/n)")
        if more_recommendations.lower() == 'n':
            print("Here are the ingredients for your selected recipe:")
            ingredients_list = get_recipe_ingredients(liked_item)
            for i, ing in enumerate(ingredients_list):
                print(f"{i+1}. {ing}")
            return


if __name__ == '__main__':
    main()


Enter user id:286566
Here are your recommendations:
1. taco seasoning   budget friendly seasoning for tacos  burritos
2. cream cheese cinnamon crescents
3. yummy crispy baked fish
4. uncle bill s fried potatoes and onions
5. crispy cheese cauliflower
Enter the number of the recipe you liked, or '0' to exit:
4
Would you like more recommendations? (y/n)y
Here are your recommendations:
1. taco seasoning   budget friendly seasoning for tacos  burritos
2. cream cheese cinnamon crescents
3. yummy crispy baked fish
4. crispy cheese cauliflower
5. microwave lemon garlic broccoli
Enter the number of the recipe you liked, or '0' to exit:
4
Would you like more recommendations? (y/n)y
Here are your recommendations:
1. taco seasoning   budget friendly seasoning for tacos  burritos
2. cream cheese cinnamon crescents
3. yummy crispy baked fish
4. uncle bill s fried potatoes and onions
5. microwave lemon garlic broccoli
Enter the number of the recipe you liked, or '0' to exit:
1
Would you like more re